### [User Input Required] Read CSV Source with Autoloader.

Autoloader Options for CSV: https://docs.databricks.com/en/ingestion/auto-loader/options.html#csv-options

Common Autoloader Options: https://docs.databricks.com/en/ingestion/auto-loader/options.html#common-auto-loader-options

In [0]:

from pipelines.shared_utils.autoloader_helper import generated_autoloader_schema_path

# [User Input Required] Set the ingest location.
ingest_location = "dbfs:/databricks-datasets/nyctaxi/tripdata/yellow"

# Auto-Generate Schema Location Based on Ingest Location
autoloader_schema_location = generated_autoloader_schema_path(ingest_location)
print("Autoloader Schema Location: " +autoloader_schema_location)

# [User Input Required] Configure Autoloader settings
df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.schemaLocation", autoloader_schema_location) # Required
    # [Input Required] Common Autoloader Settings
    .option("cloudFiles.includeExistingFiles", "true")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .option("cloudFiles.maxFilesPerTrigger", "1000")
    # [Input Required] Error Handling Settings
    .option("ignoreCorruptFiles", "false")
    # [Input Required] CSV Autoloader Settings
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("inferSchema","true")
    .load(ingest_location)
)

Autoloader Schema Location: /Volumes/gshen_etl_framework_dev/autoloader/schemas/ZGJmczovZGF0YWJyaWNrcy1kYXRhc2V0cy9ueWN0YXhpL3RyaXBkYXRhL3llbGxvdw==


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:430)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:430)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

%md
### [User Input Required] Optional Transformations

In [0]:

df.createOrReplaceTempView("tmp_view")

In [0]:
%sql
SELECT *
FROM
tmp_view

### [User Input Required] Write Data to the Lake

In [0]:
from pipelines.shared_utils.autoloader_delta_writer import AutoloaderDeltaWriter
from pipelines.base_class.data_class import WriteMode, TriggerMode


# [User Input Required] Result DataFrame
output_df = _sqldf

# [User Input Required] Target Location
datalake_location: str = ""
uc_namespace: str = ""

# [User Input Required] Configs
write_mode = WriteMode.APPEND
trigger_mode = TriggerMode.BATCH

# [User Input Required]  Instantiate writer classs instance
csv_writer: AutoloaderDeltaWriter = AutoloaderDeltaWriter(
    df=output_df,
    write_mode=write_mode,
    trigger_mode=trigger_mode,
    data_lake_path=datalake_location,
    uc_namespace=uc_namespace,
)

# [User Input Required] Write data.
csv_writer.write_append()

# Other options include
# csv_writer.write_overwrite()
# csv_writer.write_scd_1()
# csv_writer.write_scd_2()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1675829028346393>, line 17
     14 trigger_mode = ""
     16 # Instantiate writer classs instance
---> 17 csv_writer: AutoloaderDeltaWriter = AutoloaderDeltaWriter()

TypeError: Can't instantiate abstract class AutoloaderDeltaWriter with abstract methods _DeltaWriter__add_hash_cols, _DeltaWriter__generate_streaming_checkpoint_path, _DeltaWriter__read_sink_delta_path, _DeltaWriter__reset_streaming_checkpoint_path, write_append, write_overwrite, write_scd_1, write_scd_2